In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%%bash

lscpu
nvidia-smi

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) CPU @ 2.00GHz
Stepping:            3
CPU MHz:             2000.138
BogoMIPS:            4000.27
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            39424K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_si

In [ ]:
%%bash
sudo add-apt-repository ppa:gladky-anton/lammps
sudo add-apt-repository ppa:openkim/latest
sudo apt update

 
 More info: https://launchpad.net/~gladky-anton/+archive/ubuntu/lammps
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,040 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://security.ubunt

In [ ]:
%%bash
sudo apt-get install lammps-stable lammps-stable-doc lammps-stable-data openkim-models

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-mathjax javascript-common libjs-jquery libjs-mathjax libjs-underscore
  libkim-api2 libvoro++1
Suggested packages:
  apache2 | lighttpd | httpd fonts-mathjax-extras fonts-stix libjs-mathjax-doc
The following NEW packages will be installed:
  fonts-mathjax javascript-common lammps-stable lammps-stable-data
  lammps-stable-doc libjs-jquery libjs-mathjax libjs-underscore libkim-api2
  libvoro++1 openkim-models
0 upgraded, 11 newly installed, 0 to remove and 20 not upgraded.
Need to get 123 MB of archives.
After this operation, 503 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/openkim/latest/ubuntu bionic/main amd64 libkim-api2 amd64 2.2.1~202012140507~ubuntu18.04.1 [523 kB]
Get:2 http

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 11.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [9]:
%%bash
sudo mpirun --allow-run-as-root -np 2 ./hello_world

--------------------------------------------------------------------------
mpirun was unable to launch the specified application as it could not access
or execute an executable:

Executable: ./hello_world
Node: 007c58c9d600

while attempting to start process rank 0.
--------------------------------------------------------------------------
2 total processes failed to start


CalledProcessError: ignored

/bin/bash: -n: command not found


In [10]:
%%writefile MPI_Type_indexed.c
 #include "mpi.h"
   #include <stdio.h>
   #define NELEMENTS 32

   main(int argc, char *argv[])  {
   int numtasks, rank, source=0, dest, tag=1, i;
   int blocklengths[2], displacements[2];
   float a[16] = {1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0};
   float b[NELEMENTS]; 

   MPI_Status stat;
   MPI_Datatype indextype;   // required variable

   MPI_Init(&argc,&argv);
   MPI_Comm_rank(MPI_COMM_WORLD, &rank);
   MPI_Comm_size(MPI_COMM_WORLD, &numtasks);
   double t1, t2; 
   t1 = MPI_Wtime();
   blocklengths[0] = 4; /*take 4 elements from the array*/
   blocklengths[1] = 2; /*take 2 elemnets from the array*/
   displacements[0] = 5;/*start from the element index 5 the first block that is 6.0 */
   displacements[1] = 12;/*start from the element index 12  the first block that is 13.0 */
   
   // create indexed derived data type
   MPI_Type_indexed(2, blocklengths, displacements, MPI_FLOAT, &indextype);
   MPI_Type_commit(&indextype);

   if (rank == 0) {
     for (i=0; i<numtasks; i++) 
      // task 0 sends one element of indextype to all tasks
        MPI_Send(a, 1, indextype, i, tag, MPI_COMM_WORLD);
     }

   // all tasks receive indextype data from task 0
   MPI_Recv(b, NELEMENTS, MPI_FLOAT, source, tag, MPI_COMM_WORLD, &stat);
   t2 = MPI_Wtime(); 
   printf( "Elapsed time is %f\n", t2 - t1 ); 
   printf("rank= %d  b= %3.1f %3.1f %3.1f %3.1f %3.1f %3.1f\n",
          rank,b[0],b[1],b[2],b[3],b[4],b[5]);
   
   // free datatype when done using it
   MPI_Type_free(&indextype);
   MPI_Finalize();
   }

Writing MPI_Type_indexed.c


In [11]:
%%bash
mpicc MPI_Type_indexed.c -o MPI_Type_indexed

MPI_Type_indexed.c:5:4: warning: return type defaults to ‘int’ [-Wimplicit-int]
    main(int argc, char *argv[])  {
    ^~~~


In [12]:
!mpirun -n 2 --allow-run-as-root MPI_Type_indexed

Elapsed time is 0.000036
rank= 0  b= 6.0 7.0 8.0 9.0 13.0 14.0
Elapsed time is 0.000142
rank= 1  b= 6.0 7.0 8.0 9.0 13.0 14.0


In [13]:
%%writefile MPI_Type_contiguous.c
#include "mpi.h"
#include <stdio.h>
 
int main(int argc, char *argv[])
{
    int myrank;
    MPI_Status status;
    MPI_Datatype type;
    int buffer[800];
 
    MPI_Init(&argc, &argv);
 
    MPI_Type_contiguous( 800, MPI_CHAR, &type );
    double t1, t2; 
    t1 = MPI_Wtime();
    MPI_Type_commit(&type);
    
    MPI_Comm_rank(MPI_COMM_WORLD, &myrank);
 
    if (myrank == 0)
    {
        MPI_Send(buffer, 1, type, 1, 123, MPI_COMM_WORLD);
        t2 = MPI_Wtime(); 
   printf( "Elapsed time is %f\n", t2 - t1 );
    }
    else if (myrank == 1)
    {
        MPI_Recv(buffer, 1, type, 0, 123, MPI_COMM_WORLD, &status);
        t2 = MPI_Wtime(); 
   printf( "Elapsed time is %f\n", t2 - t1 );
    }
 
    MPI_Finalize();
    return 0;
}

Writing MPI_Type_contiguous.c


In [14]:
%%bash
mpicc MPI_Type_contiguous.c -o MPI_Type_contiguous

In [15]:
!mpirun -n 2 --allow-run-as-root MPI_Type_contiguous

Elapsed time is 0.000021
Elapsed time is 0.000034


In [17]:
%%writefile MPI_Type_vector.c
#include "mpi.h"
#include <stdio.h>
#include <stdlib.h>
#define SIZE 2


int main (int argc, char *argv[])
{
    int numtasks, rank, source=0, dest, tag=1, i;
    float a[SIZE][SIZE] =
    {10000.0, 20000.0, 
     30000.0, 40000.0, };
    float b[SIZE];

    MPI_Status stat;
     
    MPI_Datatype columntype;

    MPI_Init(&argc,&argv);
    
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    MPI_Comm_size(MPI_COMM_WORLD, &numtasks);
    
    double t1, t2; 
    t1 = MPI_Wtime();
    MPI_Type_vector(SIZE/*num of element in a column*/,
                     1 /*one element for row*/, 
                    SIZE /*take an element each 4*/, MPI_FLOAT, &columntype);
                    
    MPI_Type_commit(&columntype);
    

    if (numtasks == SIZE) {
    if (rank == 0) {
        for (i=0; i<numtasks; i++)
       
        MPI_Send(&a[0][i], 1, columntype, i, tag, MPI_COMM_WORLD);
            }

    MPI_Recv(b, SIZE, MPI_FLOAT, source, tag, MPI_COMM_WORLD, &stat);
     
    printf("rank= %d  b= %3.1f %3.1f\n",
            rank,b[0],b[1]);
            t2 = MPI_Wtime(); 
            printf( "Elapsed time is %f\n", t2 - t1 );
    }
    else
    printf("Must specify %d processors. Terminating.\n",SIZE);

    MPI_Type_free(&columntype);
    MPI_Finalize();
}

Overwriting MPI_Type_vector.c


In [18]:
%%bash
mpicc MPI_Type_vector.c -o MPI_Type_vector

In [19]:
!mpirun -n 2 --allow-run-as-root MPI_Type_vector

rank= 0  b= 10000.0 30000.0
Elapsed time is 0.000083
rank= 1  b= 20000.0 40000.0
Elapsed time is 0.000067


In [20]:
%%writefile MPI_Type_index.c
#include "mpi.h"
   #include <stdio.h>
   #define NELEM 15

   main(int argc, char *argv[])  {
   int numtasks, rank, source=0, dest, tag=1, i;

   typedef struct {
     float x, y, z;
     float velocity;
     int  n, type;
     }          Particle;
   Particle     p[NELEM], particles[NELEM];
   MPI_Datatype particletype, oldtypes[2];   // required variables
   int          blockcounts[2];

   // MPI_Aint type used to be consistent with syntax of
   // MPI_Type_extent routine
   MPI_Aint    offsets[2], lb, extent;

   MPI_Status stat;

   MPI_Init(&argc,&argv);
   double t1, t2; 
    t1 = MPI_Wtime();
   MPI_Comm_rank(MPI_COMM_WORLD, &rank);
   MPI_Comm_size(MPI_COMM_WORLD, &numtasks);
 
   // setup description of the 4 MPI_FLOAT fields x, y, z, velocity
   offsets[0] = 0;
   oldtypes[0] = MPI_FLOAT;
   blockcounts[0] = 4;

   // setup description of the 2 MPI_INT fields n, type
   // need to first figure offset by getting size of MPI_FLOAT
   MPI_Type_get_extent(MPI_FLOAT, &lb, &extent);
   offsets[1] = 4 * extent;
   oldtypes[1] = MPI_INT;
   blockcounts[1] = 2;

   // define structured type and commit it
   MPI_Type_create_struct(2, blockcounts, offsets, oldtypes, &particletype);
   MPI_Type_commit(&particletype);

   // task 0 initializes the particle array and then sends it to each task
   if (rank == 0) {
     for (i=0; i<NELEM; i++) {
        particles[i].x = i * 1.0;
        particles[i].y = i * -1.0;
        particles[i].z = i * 1.0; 
        particles[i].velocity = 0.25;
        particles[i].n = i;
        particles[i].type = i % 2; 
        }
     for (i=0; i<numtasks; i++) 
        MPI_Send(particles, NELEM, particletype, i, tag, MPI_COMM_WORLD);
     }
 
   // all tasks receive particletype data
   MPI_Recv(p, NELEM, particletype, source, tag, MPI_COMM_WORLD, &stat);

   printf("rank= %d   %3.2f %3.2f %3.2f %3.2f %d %d\n", rank,p[3].x,
        p[3].y,p[3].z,p[3].velocity,p[3].n,p[3].type);
        t2 = MPI_Wtime();
        printf( "Elapsed time is %f\n", t2 - t1 );

   // free datatype when done using it
   MPI_Type_free(&particletype);
   MPI_Finalize();
   }

Writing MPI_Type_index.c


In [21]:
!mpicc MPI_Type_struct.c -o MPI_Type_struct

gcc: error: MPI_Type_struct.c: No such file or directory


In [22]:
!mpirun -n 2 --allow-run-as-root MPI_Type_struct

--------------------------------------------------------------------------
mpirun was unable to find the specified executable file, and therefore
did not launch the job.  This error was first reported for process
rank 0; it may have occurred for other processes as well.

NOTE: A common cause for this error is misspelling a mpirun command
      line parameter option (remember that mpirun interprets the first
      unrecognized command line token as the executable).

Node:       007c58c9d600
Executable: MPI_Type_struct
--------------------------------------------------------------------------
2 total processes failed to start


In [23]:
%%writefile MPI_Bcast.c



#include <stdio.h>
#include "mpi.h"

int main(int argc, char** argv) {

        int rank;
        int data=0;
        const int root=0;

        MPI_Init(&argc, &argv);
        double t1, t2; 
t1 = MPI_Wtime();
        MPI_Comm_rank(MPI_COMM_WORLD, &rank);

        if(rank == root) {
           data = 1000;
        }

        

        /* everyone calls bcast, data is taken from root and ends up in everyone's data */
        MPI_Bcast(&data, 1, MPI_INT, root, MPI_COMM_WORLD);
        t2 = MPI_Wtime();
printf( "Elapsed time is %f\n", t2 - t1 );
 
        printf("[Process %d]: After Bcast, data is %d\n", rank, data);

        MPI_Finalize();
        return 0;
}

Writing MPI_Bcast.c


In [24]:
!mpicc MPI_Bcast.c -o MPI_Bcast

In [25]:
!mpirun -n 2 --allow-run-as-root MPI_Bcast

Elapsed time is 0.000024
[Process 0]: After Bcast, data is 1000
Elapsed time is 0.000030
[Process 1]: After Bcast, data is 1000


In [26]:
%%writefile MPI_Bcast_String.c

#include "mpi.h"
#include <stdlib.h>
#include <stdio.h>
#include <string.h>

#define ROOT 0
#define NUM_REPS 5
#define NUM_SIZES 3

int main( int argc, char **argv)
{
    int *buf;
    int i, rank, reps, n;
    int bVerify = 1;
    int sizes[NUM_SIZES] = { 100, 64*1024, 128*1024 };
    int num_errors=0, tot_errors;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    double t1, t2; 
t1 = MPI_Wtime();
    if (argc > 1)
    {
        if (strcmp(argv[1], "-novalidate") == 0 || strcmp(argv[1], "-noverify") == 0)
            bVerify = 0;
    }
    buf = (int *) malloc(sizes[NUM_SIZES-1]*sizeof(int));
    memset(buf, 0, sizes[NUM_SIZES-1]*sizeof(int));
    for (n=0; n<NUM_SIZES; n++)
    {
        if (rank == ROOT)
        {
            printf("bcasting %d MPI_INTs %d times\n", sizes[n], NUM_REPS);
            fflush(stdout);
        }
        for (reps=0; reps < NUM_REPS; reps++)
        {
            if (bVerify)
            {
                if (rank == ROOT)
                {
                    for (i=0; i<sizes[n]; i++)
                    {
                        buf[i] = 1000000 * (n * NUM_REPS + reps) + i;
                    }
                }
                else
                {
                    for (i=0; i<sizes[n]; i++)
                    {
                        buf[i] = -1 - (n * NUM_REPS + reps);
                    }
                }
            }
            MPI_Bcast(buf, sizes[n], MPI_INT, ROOT, MPI_COMM_WORLD);
            if (bVerify)
            {
                num_errors = 0;
                for (i=0; i<sizes[n]; i++)
                {
                    if (buf[i] != 1000000 * (n * NUM_REPS + reps) + i)
                    {
                        num_errors++;
                        if (num_errors < 10)
                        {
                           
 
                            printf("Error: Rank=%d, n=%d, reps=%d, i=%d, buf[i]=%d expected=%d\n", rank, n, reps, i, buf[i],
                                    1000000 * (n * NUM_REPS + reps) +i);
                            fflush(stdout);
                        }
                    }
                }
                if (num_errors >= 10)
                {
                   
 
                    printf("Error: Rank=%d, num_errors = %d\n", rank, num_errors);
                    fflush(stdout);
                }
            }
        }
    }
 t2 = MPI_Wtime();
printf( "Elapsed time is %f\n", t2 - t1 );
 
    MPI_Reduce( &num_errors, &tot_errors, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD );
    if (rank == 0 && tot_errors == 0)
        
    fflush(stdout);
    free(buf);
    MPI_Finalize();
    return 0;
}

Writing MPI_Bcast_String.c


In [ ]:
!mpicc MPI_Bcast_String.c -o MPI_Bcast_String

In [28]:
!mpirun -n 2 --allow-run-as-root MPI_Bcast_String

bcasting 100 MPI_INTs 5 times
bcasting 65536 MPI_INTs 5 times
bcasting 131072 MPI_INTs 5 times
Elapsed time is 0.010887
Elapsed time is 0.010849


In [29]:
%%writefile MPI_Type_indexFIS.c

 #include "mpi.h"
   #include <stdio.h>
   #define NELEMENTS 6

   main(int argc, char *argv[])  {
   int numtasks, rank, source=0, dest, tag=1, i;
   int blocklengths[2], displacements[2];
   float a[16] = {1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0};
   float b[NELEMENTS]; 
   char city[7] = "Athens";

   MPI_Status stat;
   MPI_Datatype indextype;   // required variable

   MPI_Init(&argc,&argv);
   MPI_Comm_rank(MPI_COMM_WORLD, &rank);
   MPI_Comm_size(MPI_COMM_WORLD, &numtasks);
   double t1, t2; 
   t1 = MPI_Wtime();
   blocklengths[0] = 4; 
   blocklengths[1] = 2; 
   displacements[0] = 5;
   displacements[1] = 12;
   
   // create indexed derived data type
   MPI_Type_indexed(2, blocklengths, displacements, MPI_FLOAT, &indextype,city);
   MPI_Type_commit(&indextype);

   if (rank == 0) {
     for (i=0; i<numtasks; i++) 
      // task 0 sends one element of indextype to all tasks
        MPI_Send(a, 1, indextype, i, tag, MPI_COMM_WORLD);
     }

   // all tasks receive indextype data from task 0
   MPI_Recv(b, NELEMENTS, MPI_FLOAT, source, tag, MPI_COMM_WORLD, &stat,city);
   t2 = MPI_Wtime(); 
   printf( "Elapsed time is %f\n", t2 - t1 ); 
   printf("rank= %d  b= %3.1f %3.1f %3.1f %3.1f %3.1f %3.1f\n",
          rank,b[0],b[1],b[2],b[3],b[4],b[5]);
   
   // free datatype when done using it
   MPI_Type_free(&indextype);
   MPI_Finalize();
   }

Writing MPI_Type_indexFIS.c


In [30]:
!mpicc MPI_Type_indexFIS.c -o MPI_Type_indexFIS

MPI_Type_indexFIS.c:6:4: warning: return type defaults to ‘int’ [-Wimplicit-int]
    main(int argc, char *argv[])  {
    ^~~~
MPI_Type_indexFIS.c: In function ‘main’:
MPI_Type_indexFIS.c:27:4: error: too many arguments to function ‘MPI_Type_indexed’
    MPI_Type_indexed(2, blocklengths, displacements, MPI_FLOAT, &indextype,city);
    ^~~~~~~~~~~~~~~~
In file included from MPI_Type_indexFIS.c:2:0:
/usr/lib/x86_64-linux-gnu/openmpi/include/mpi.h:1808:20: note: declared here
 OMPI_DECLSPEC  int MPI_Type_indexed(int count, const int array_of_blocklengths[],
                    ^~~~~~~~~~~~~~~~
MPI_Type_indexFIS.c:37:4: error: too many arguments to function ‘MPI_Recv’
    MPI_Recv(b, NELEMENTS, MPI_FLOAT, source, tag, MPI_COMM_WORLD, &stat,city);
    ^~~~~~~~
In file included from MPI_Type_indexFIS.c:2:0:
/usr/lib/x86_64-linux-gnu/openmpi/include/mpi.h:1641:20: note: declared here
 OMPI_DECLSPEC  int MPI_Recv(void *buf, int count, MPI_Datatype datatype, int source,
                    ^~~~~

In [31]:
!mpirun -n 2 --allow-run-as-root MPI_Type_indexFIS

--------------------------------------------------------------------------
mpirun was unable to find the specified executable file, and therefore
did not launch the job.  This error was first reported for process
rank 0; it may have occurred for other processes as well.

NOTE: A common cause for this error is misspelling a mpirun command
      line parameter option (remember that mpirun interprets the first
      unrecognized command line token as the executable).

Node:       007c58c9d600
Executable: MPI_Type_indexFIS
--------------------------------------------------------------------------
2 total processes failed to start


In [32]:
%%writefile MPI_Allgatherv.c
#include <stdio.h>
#include <mpi.h>     

int main(int argc, char ** argv) 
{

   int nprocs; /* Number of processes    */
   int rank;   /* My process rank        */

   /* Start up MPI */
   MPI_Init(&argc, &argv); 

   /* Get the number of processes */
   MPI_Comm_size(MPI_COMM_WORLD, &nprocs); 
double t1, t2; 
   t1 = MPI_Wtime();
   
   MPI_Comm_rank(MPI_COMM_WORLD, &rank); 

   int n = 5;
   float xx[n];

   int i;
   for (i=0;i<n;i++)
   {
      if ( rank == 0 )
         xx[i] = (float)i;
      else
         xx[i] = -1.0*(float)i;

      printf("BEFORE***   rank: %d i: %d xx[i]: %f\n",rank,i,xx[i]);
   }

   int scounts[2];
   scounts[0] = 3; // slice one is 3 pieces of data
   scounts[1] = 2; // slice two is 2 pieces of data

   int displs[2];
   displs[0] = 0; // slice one starts at array element 0
   displs[1] = 3; // slice two starts at array element 3

   MPI_Allgatherv(MPI_IN_PLACE,0,MPI_FLOAT,xx,scounts,displs,MPI_FLOAT,MPI_COMM_WORLD);

   for (i=0;i<n;i++)
   t2 = MPI_Wtime(); 
   printf( "Elapsed time is %f\n", t2 - t1 ); 
      printf("AFTER***  rank: %d i: %d xx[i]: %f\n",rank,i,xx[i]);
   
   /* Shut down MPI */
   MPI_Finalize(); 

   return 0;

}  

Overwriting MPI_Allgatherv.c


In [33]:
!mpicc MPI_Allgatherv.c -o MPI_Allgatherv

In [34]:
!mpirun -n 2 --allow-run-as-root MPI_Allgatherv

BEFORE***   rank: 0 i: 0 xx[i]: 0.000000
BEFORE***   rank: 0 i: 1 xx[i]: 1.000000
BEFORE***   rank: 0 i: 2 xx[i]: 2.000000
BEFORE***   rank: 0 i: 3 xx[i]: 3.000000
BEFORE***   rank: 0 i: 4 xx[i]: 4.000000
BEFORE***   rank: 1 i: 0 xx[i]: -0.000000
BEFORE***   rank: 1 i: 1 xx[i]: -1.000000
BEFORE***   rank: 1 i: 2 xx[i]: -2.000000
BEFORE***   rank: 1 i: 3 xx[i]: -3.000000
BEFORE***   rank: 1 i: 4 xx[i]: -4.000000
Elapsed time is 0.000243
AFTER***  rank: 0 i: 5 xx[i]: 0.000000
Elapsed time is 0.000255
AFTER***  rank: 1 i: 5 xx[i]: 0.000000
